# Homework 3

## FINM 37500 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

In [30]:
import pandas as pd
import datetime as dt
import helper_functions
import ficcvol, binomial

# 1. Treasury Futures and Cheapest-to-Deliver

The file `data/fut_bond_data_FVU3_2023-04-21.xlsx` has market data on the following:
* 5-year Treasury future, expiring September 2023
* The specifications of the deliverable treasury bonds

Market quotes are provided on the futures contract and the bond prices. These will be useful for some of the analysis questions, but you do not need them for your models as you are provided a BDT tree which is fit to swaps and caps. See below for more details on this BDT model.

Suppose the present date is `2023-04-21`.

### BDT Model

In this problem you will make use of a BDT modeled binomial tree.

To save you some time, you are provided the parameters of a BDT tree fit to both swaps and caps.
* Use the file `bdt_params_freq52_2023-04-21.xlsx`
* With these $\sigma$ and $\theta$ parameters, you should be able to build a BDT tree with $T=5$ and $dt=1/52$.

#### Note
If interested in how this was done, find the data and files used to get these parameters. In particular, 
* The market quotes interpolated to weekly frequency: `cap_curves_2023-04-21_freq_52.xlsx`.
* The file to estimate the model is `Parameterize BDT.ipynb`.

## 1.1 Trading Bonds

Give brief answers to these based on the market quotes provided,
### 1.1.1
Calculate the 
* gross basis
* carry
* net basis for each bond

### 1.1.2
Which bond seems most likely to be CTD?

### 1.1.3
If you were required to put on a position today
* long one of the bonds
* short the future

which would you choose based on the data provided in the spreadsheet?

In [2]:
futures = pd.read_excel("/Users/zhenyuwang/Library/Mobile Documents/com~apple~CloudDocs/U Chicago/FI Derivatives/fut_bond_data_FVU3_2023-04-21.xlsx", sheet_name = 'future')
futures_price = futures.loc[1,'FVU3 Comdty']
futures_ttm = futures.loc[4,'FVU3 Comdty']/360
futures

,field,FVU3 Comdty
0,last_update_dt,2023-04-21 00:00:00
1,px_last,109.789062
2,last_tradeable_dt,2023-09-29 00:00:00
3,fut_dlv_dt_last,2023-10-04 00:00:00
4,fut_days_expire,159
5,fut_ctd,T 3.875 11/30/27
6,fut_ctd_px,100.757812
7,fut_ctd_gross_basis,-17.074348
8,fut_ctd_net_basis,1.199828


In [18]:
bonds = pd.read_excel("/Users/zhenyuwang/Library/Mobile Documents/com~apple~CloudDocs/U Chicago/FI Derivatives/fut_bond_data_FVU3_2023-04-21.xlsx", sheet_name = 'bonds')
bonds['future_px'] = futures_price
bonds['future_ttm'] = futures_ttm
bonds['convereted_future_px'] = bonds['future_px'] * bonds['conversion']
bonds['gross_basis'] = bonds['px_last'] - bonds['convereted_future_px']
bonds['carry'] = bonds['cpn'] - bonds['repo_reporate']
bonds['net_basis'] = bonds['gross_basis'] - bonds['carry'] * bonds['future_ttm']
bonds['future_dlv'] = futures.loc[3,'FVU3 Comdty']
bonds['TTM_at_dlv'] = (bonds['maturity'] - bonds['future_dlv'])/dt.timedelta(365.25)
bonds
#bonds['conversion_factor'] = bonds[['px_last','cpn','TTM_at_dlv']].apply(helper_functions.cashflow_calc())

,ticker,last_update_dt,px_last,maturity,days_to_mty,cpn,nxt_cpn_dt,days_to_next_coupon,int_acc,accrued_days_between_cpn_dates,...,repo_reporate,conversion,future_px,future_ttm,convereted_future_px,gross_basis,carry,net_basis,future_dlv,TTM_at_dlv
0,91282CFZ Govt,2023-04-21,100.757812,2027-11-30,1681,3.875,2023-05-31,37,1.543613,182,...,4.815,0.9226,109.789062,0.441667,101.291389,-0.533577,-0.940,-0.118410,2023-10-04,4.156057
1,91282CGC Govt,2023-04-21,100.750000,2027-12-31,1712,3.875,2023-06-30,67,1.220304,181,...,4.815,0.9212,109.789062,0.441667,101.137684,-0.387684,-0.940,0.027482,2023-10-04,4.240931
2,91282CGH Govt,2023-04-21,99.195312,2028-01-31,1743,3.500,2023-07-31,98,0.802486,181,...,4.815,0.9058,109.789062,0.441667,99.446933,-0.251620,-1.315,0.329171,2023-10-04,4.325804
3,91282CGP Govt,2023-04-21,101.484375,2028-02-29,1772,4.000,2023-08-31,129,0.597826,184,...,4.815,0.9234,109.789062,0.441667,101.379220,0.105155,-0.815,0.465113,2023-10-04,4.405202
4,91282CGT Govt,2023-04-21,99.828125,2028-03-31,1803,3.625,2023-09-30,159,0.237705,183,...,4.815,0.9075,109.789062,0.441667,99.633574,0.194551,-1.190,0.720134,2023-10-04,4.490075


In [22]:

def price_calc_flat_discount(price, coupon, years_to_maturity, discount_rate):
    cash_flow = [100+coupon/2]
    cash_time = [years_to_maturity]
    time_countdown = years_to_maturity
    price = (100+coupon/2)*(1+discount_rate/2)**(-2*years_to_maturity)
    while time_countdown-0.5 > 1/365.25:
        time_countdown -= 0.5
        cash_flow += [coupon/2]
        cash_time += [time_countdown]
        price += coupon/2*(1+discount_rate/2)**(-2*time_countdown)
    return price



In [23]:
price_calc_flat_discount(100.757812,3.875,4.156057,0.06)

93.61144831761911

## 1.2 Conversion Factors

Calculate the conversion factor for each bond. Report it to `6` decimal places.

Do they match the conversion factor provided by Bloomberg?

## 1.3 BDT Tree

Report the number of steps for
* each bond's maturity
* the futures contract expiration

Build the interest-rate tree and display it.

In [33]:
curves = pd.read_excel("/Users/zhenyuwang/Library/Mobile Documents/com~apple~CloudDocs/U Chicago/FI Derivatives/cap_curves_2023-04-21_freq_52.xlsx")
capcurves = ficcvol.flat_to_forward_vol(curves)
curves

/Users/zhenyuwang/Library/Mobile Documents/com~apple~CloudDocs/U Chicago/FI Derivatives/ficcvol.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (1/sigT) * np.log(fwd/strike) + .5*sigT
/Users/zhenyuwang/Library/Mobile Documents/com~apple~CloudDocs/U Chicago/FI Derivatives/ficcvol.py:23: RuntimeWarning: divide by zero encountered in divide
  d1 = (1/sigT) * np.log(fwd/strike) + .5*sigT
/Users/zhenyuwang/Library/Mobile Documents/com~apple~CloudDocs/U Chicago/FI Derivatives/ficcvol.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (1/sigT) * np.log(fwd/strike) + .5*sigT
/Users/zhenyuwang/Library/Mobile Documents/com~apple~CloudDocs/U Chicago/FI Derivatives/ficcvol.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (1/sigT) * np.log(fwd/strike) + .5*sigT
/Users/zhenyuwang/Library/Mobile Documents/com~apple~CloudDocs/U Chicago/FI Derivatives/ficcvol.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  d1

,tenor,swap rates,discounts,forwards,flat vols
0,0.019231,0.048917,0.999060,NaN,NaN
1,0.038462,0.049131,0.998113,0.049346,0.234111
2,0.057692,0.049331,0.997159,0.049732,0.237132
3,0.076923,0.049517,0.996200,0.050076,0.240152
4,0.096154,0.049690,0.995236,0.050379,0.243173
...,...,...,...,...,...
515,9.923077,0.032862,0.721816,0.032632,0.359418
516,9.942308,0.032861,0.721363,0.032640,0.359252
517,9.961538,0.032861,0.720910,0.032648,0.359093
518,9.980769,0.032860,0.720458,0.032656,0.358944


In [32]:
quotes = curves['discounts'] * 100
sigmas = capcurves['fwd vols']
sigmas.iloc[0] = sigmas.iloc[1]

theta, ratetree = binomial.estimate_theta(sigmas, quotes)

Completed: 0.2%


KeyError: -1

## 1.4 Bond Pricing

Use the tree to price each bond. Report
* time-0 dirty and clean price of each bond
* terminal (clean) value of each bond at futures expiration, for each state of the tree.

Thus, to report the terminal values you will need to grab the expiration column of each bond's (clean) pricing tree and adjust (inflate) it for the conversion factor.

## 1.5 CTD

Use your terminal values calculated above to state which bond is CTD in each interest-rate state (at this expiration node.) 

Report the duration of each bond (as of today's price, not recomputed for the interest-rate nodes.) Do you see a relationship between the time-0 duration and the at-expiration CTD?

## 1.6 Futures Price

Model the futures price with the tree approach.
* Use the CTD terminal value for each rate.
* Step backward through the tree.

As you step backward remember that for a futures contract
* no discounting by the riskfree rate
* the futures contract has no capital requirement and thus an expected P&L of zero under this measure.

Thus, each node is the simple average of the two nodes at the following step.

**Report the futures price.**

### Compare
How does it compare to 
* the quoted futures price
* the modeled bond prices

## 1.7 Early Delivery
**Optional**
Above we modeled the terminal value at the futures expiration. Now consider if early delivery would be better.

Which periods in the tree are eligible to deliver based on the parameters of the 5-year futures contract? 

Based on your model, does it make sense to deliver early in any of the nodes of the tree?

## 1.8 Option-Adjusted Spread
**Optional**

Calculate and report the option-adjusted spread (OAS) for the future.

Note that you
* do NOT need to recalculate the bond prices
* will simply add a constant rate (at every node) for discounting the futures price in the previous problem.

What does the OAS indicate?

***

# 2. Fed Funds Futures

The file `data/fedfutures_2023-04-21.xlsx`jj has market data on the following:
* Fed Fund Futures Chain out 18 months.
* Dates of upcoming Fed meetings (approximated in 2024.)
* Spot Fed Funds data
* Prices of the futures chain on a historic date.

Suppose the present date is `2023-04-21`.

## 2.1 Chart the Fed Futures Rates

Chart the Fed Funds curve at
* the present date
* the historic date

Note that you are charting the implied Fed Funds Futures *rate*, not price.

Comment on how today's **open interest** varies across the chain.

## 2.2 Extracting the Expected Path of Fed Funds Rates

The Fed has a great deal of control over the Fed Funds Rate. We simplify by assuming the Fed 
* sets the rate exactly at its list of meeting dates.
* does not change the rate betweeen meeting dates.

Use the present data to calculate--and plot--the implied set of expected Fed Funds rates as of each meeting date.

#### Note
One (minor) assumption:
* Consider months, $t$, where there is a meeting, but such that in month $t+1$ there is no meeting.
* There will be two reasonable ways to extract the expected fed funds rate:
    1. Use the futures rate from the $t+1$-contract
    2. Calculate the implied rate for the remainder of month $t$, knowing the expected rate at the end of month $t+1$.
* These are both reasonable and will likely not differ much.
* Here, use the simpler method #1--that is, for months with no meeting in the following month, the calculation is very simple.

## 2.3 Compare to the Historic Curve

Use the price data in the historic tab to extract the expectations at the previous date.
* Note that you do not need to "bootstrap" up from the historic date to the current date. 
* There was no meeting in the current month, so its futures price is enough to get started.

Compare this to the answer in the previous problem, for the current data.

## 2.4 Analyzing the Expected Path

These questions are both conceptual--no calculation required.

### 2.4.1
Conceptually, is the path extracted above the **expected path**? In what sense is it or is it not?

### 2.4.2

Probability Distributions

The implied path above is not representative of any single actual path of Fed rates, which are typically changed by 25bps at a time.

Conceptually, what would you need to make probability statements about the Fed moving rates up/down by 25bps on any given meeting date? For instance, as seen in the `probabilities` tab of the [CME FedWatch Tool](https://www.cmegroup.com/markets/interest-rates/cme-fedwatch-tool.html)?

Do not quantitatively solve this--just a conceptual answer is fine.

***